In [1]:
import re
import pdfminer
from subprocess import check_output
from bs4 import BeautifulSoup
from collections import defaultdict

In [2]:
# install pdfminer with this:
# pip3 install pdfminer.six

In [3]:
# run this in shell to grab file:
# curl "https://madison.legistar.com/View.ashx?M=A&ID=782498&GUID=95864EAA-10C1-4866-AA34-C134B80BA5F5" -o agenda.pdf

In [4]:
s = check_output(["pdf2txt.py", "-t", "html", "-Y", "loose", "agenda.pdf"])

In [5]:
page = BeautifulSoup(s)
text = {} # key=(x,y), val=text

for div in page.find_all("div"):
    style = div.attrs.get("style", "")
    m1 = re.search("left:(\d+)px", style)
    m2 = re.search("top:(\d+)px", style)
    if m1 and m2:
        x, y = int(m1.group(1)), int(m2.group(1))
        text[(x,y)] = div.get_text()

In [6]:
def find_near(x, y):
    for x_ in range(x-5, x+5):
        for y_ in range(y-5, y+5):
            if (x_, y_) in text:
                return text[(x_, y_)]
    return None

agenda_list = {}

for (x,y), t in text.items():
    # agenda items are roughly at x=170
    if 165 < x < 175:
        # agenda numbers are rougly at x=69, same y as text
        num = find_near(69, y)
        if num:
            agenda_list[int(num.split(".")[0])] = t

agenda_list

{3: 'Opening Comments - Mayor Satya Rhodes-Conway (10 minutes)\nMetro Route Restructure, Complete Streets, Parking Ordinance and TOD \nOverlay, BRT\n \n',
 4: 'Introduction of New Transit General Manager - Tom Lynch, Justin Stuehrenberg \n(10 minutes)\n \n',
 5: 'Adopting the Oscar Mayer Special Area Plan as a Supplement to the City of \nMadison Comprehensive Plan.\n',
 6: 'Amending the timeline for final Staff Team recommendations on parking \npolicies as adopted by RES-19-00321 (Legistar File ID 54961) on 4/16/2019.\n',
 7: 'Authorizing the Mayor and the City Clerk to enter into a project agreement with \nthe Wisconsin Department of Transportation (WisDOT) and other project \npartners to construct an interchange at USH 12/18 and County AB. (16 AD)\n',
 8: 'Metro Status and Update on Metro Restructure RFP - Mike Cechvala\n \n',
 9: 'COVID-19 Transportation Items - Tom Lynch, Sabrina Tolley\n \n',
 10: 'Wilson Street Corridor Plan \n \n',
 11: 'Vehicle Miles Traveled and Mobility Initi